In [1]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [2]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [3]:
import boto3
from decimal import Decimal
# Get the service resource.
dynamodb = boto3.resource('dynamodb', region_name='ap-northeast-2')
table = dynamodb.Table('dsnia_imagelabelsV2')
# table = dynamodb.Table('dsnia_imagelabels')

In [4]:
course_list = ['청도','포항','중문','시흥','거창','속리산','춘천']

course_ids = {
  '포항':'MGC001',
  '청도':'MGC002',
  '중문':'MGC003',
  '시흥':'MGC004',
  '거창':'MGC005',
  '속리산':'MGC006',
  '춘천':'MGC007'
}

folder_type = ['rgb','ndvi','thumb']

In [5]:
course_id = course_ids.get('청도')
course_id

'MGC002'

In [6]:
AWS_folder = os.getcwd()

b_folder = os.path.join(AWS_folder,'BACKUPS')
b_folder

src_folder = os.path.join(AWS_folder,'dataDownloaded')
src_folder

out_folder = os.path.join(AWS_folder,'filepathBackUp')
out_folder

'c:\\Users\\yc463\\Documents\\JamesLocal\\DS\\Drone\\dsmapv2\\src\\AWS\\filepathBackUp'

In [7]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError



courseid = "MGC002"
date = "20230823"


response = table.query(IndexName='courseid-id-index', KeyConditionExpression=Key('courseid').eq(courseid),  FilterExpression=Attr('date').eq(date))
alldata =  response['Items']

while 'LastEvaluatedKey' in response:
    response = table.query(IndexName='courseid-id-index', KeyConditionExpression=Key('courseid').eq(courseid), 
     ExclusiveStartKey=response['LastEvaluatedKey'] ,FilterExpression=Attr('date').eq(date) )    
    # response_ = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    alldata.extend(response['Items'])

In [8]:
len(alldata)

8319

In [9]:
def update_root(x):
  path = os.path.normpath(x)
  info = path.split(os.sep)
  index_ = [idx for idx, s in enumerate(info) if '2023' and '완' in s][0]
  return 'original/'+ ('/').join(info[index_:])

In [10]:
for jsonSinlgle in alldata:
  jsonSinlgle['json']['originalFileJPG'] = update_root(jsonSinlgle['json']['originalFileJPG'])
  jsonSinlgle['json']['originalFileGrp'] = [update_root(x) for x in jsonSinlgle['json']['originalFileGrp']]


In [13]:
alldata[3000]

{'metadata': 'MGC002202308234_마운틴1H패취류,섬머패취,켄터키,티,잡초,피막이,바랭이,민들레,파대가리_0',
 'date': '20230823',
 'area': '4_마운틴1H',
 'courseid': 'MGC002',
 'json': {'originalFileGrp': ['original/2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본_완/4_마운틴1H/2023_0823_청도그레이스_마운틴 1H_0.5m/20230823_1328-1352_패취류,섬머패취,켄터키,티,잡초,피막이,바랭이,민들레,파대가리/DJI_20230823132925_0024_MS_G.TIF',
   'original/2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본_완/4_마운틴1H/2023_0823_청도그레이스_마운틴 1H_0.5m/20230823_1328-1352_패취류,섬머패취,켄터키,티,잡초,피막이,바랭이,민들레,파대가리/DJI_20230823132925_0024_MS_NIR.TIF',
   'original/2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본_완/4_마운틴1H/2023_0823_청도그레이스_마운틴 1H_0.5m/20230823_1328-1352_패취류,섬머패취,켄터키,티,잡초,피막이,바랭이,민들레,파대가리/DJI_20230823132925_0024_MS_R.TIF',
   'original/2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본_완/4_마운틴1H/2023_0823_청도그레이스_마운틴 1H_0.5m/20230823_1328-1352_패취류,섬머패취,켄터키,티,잡초,피막이,바랭이,민들레,파대가리/DJI_20230823132925_0024_MS_RE.TIF'],
  'annotation': [],
  'labelBy': '',
  'GeoTagInfo': {'gps_altitude': Decimal('165.179'),
   'datetime_original': '2023:08:23 1

In [14]:
with table.batch_writer(overwrite_by_pkeys=['metadata', 'id']) as batch:
  for index_, row_ in enumerate(alldata):
    print(index_, row_['id'], end='\r')
    batch.put_item(
      Item= row_
    )

In [15]:



response = table.query(IndexName='courseid-id-index', KeyConditionExpression=Key('courseid').eq(courseid),  FilterExpression=Attr('date').eq(date))
data =  response['Items']

while 'LastEvaluatedKey' in response:
    response = table.query(IndexName='courseid-id-index', KeyConditionExpression=Key('courseid').eq(courseid),  ExclusiveStartKey=response['LastEvaluatedKey'] ,FilterExpression=Attr('date').eq(date) )    
    # response_ = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

In [16]:
len(data)

8319